In [22]:
import pandas as pd
import numpy as np
import datetime
import os
import re

os.chdir('/Users/arthur/Desktop/factor_investment')

def del_slash(x):
    return x[0:4]+x[5:7]+x[8:10]

def clearSentence(sentence):
    sentence = re.sub('-KY', '', sentence)
    sentence = re.sub('\*', '', sentence)
    return sentence

In [2]:
col_list = ["證券代碼", "簡稱", "年月日", "收盤價(元)", "報酬率％"]
price_df = pd.read_csv('./data/TEJ_StockPrice_20052022.csv', encoding='big5', encoding_errors='ignore', usecols=col_list)
price_df["證券代碼"] = [str(x).strip() for x in price_df["證券代碼"]]
price_df["簡稱"] = [x.strip() for x in price_df["簡稱"]]

/var/folders/w8/jwxq4stj29506b1pf67xpb000000gq/T/ipykernel_983/3197629334.py:2: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  price_df = pd.read_csv('./data/TEJ_StockPrice_20052022.csv', encoding='big5', encoding_errors='ignore', usecols=col_list)


In [3]:
with open("./data/StockValidation.csv", 'r') as file:
    validstock_df = pd.read_csv(file, sep=',', low_memory=False, index_col=0)
    validstock_df['stock_id'] = validstock_df['stock_id'].astype('str')

In [4]:
price_df = price_df[price_df.證券代碼.isin(validstock_df.stock_id)]
price_df = price_df.reset_index(drop=True)
price_df

,證券代碼,簡稱,年月日,收盤價(元),報酬率％
0,50,元大台灣50,2005/01/03,26.0412,-0.4124
1,50,元大台灣50,2005/01/04,25.6045,-1.677
2,50,元大台灣50,2005/01/05,25.2702,-1.3055
3,50,元大台灣50,2005/01/06,25.2702,0.0
4,50,元大台灣50,2005/01/07,25.0708,-0.7894
...,...,...,...,...,...
3892485,9958,世紀鋼,2022/05/10,117.5000,1.2931
3892486,9958,世紀鋼,2022/05/11,115.5000,-1.7021
3892487,9958,世紀鋼,2022/05/12,112.0000,-3.0303
3892488,9958,世紀鋼,2022/05/13,116.0000,3.5714


In [5]:
price_df['日期'] = price_df['年月日'].apply(lambda x : del_slash(x))

In [6]:
price_df

,證券代碼,簡稱,年月日,收盤價(元),報酬率％,日期
0,50,元大台灣50,2005/01/03,26.0412,-0.4124,20050103
1,50,元大台灣50,2005/01/04,25.6045,-1.677,20050104
2,50,元大台灣50,2005/01/05,25.2702,-1.3055,20050105
3,50,元大台灣50,2005/01/06,25.2702,0.0,20050106
4,50,元大台灣50,2005/01/07,25.0708,-0.7894,20050107
...,...,...,...,...,...,...
3892485,9958,世紀鋼,2022/05/10,117.5000,1.2931,20220510
3892486,9958,世紀鋼,2022/05/11,115.5000,-1.7021,20220511
3892487,9958,世紀鋼,2022/05/12,112.0000,-3.0303,20220512
3892488,9958,世紀鋼,2022/05/13,116.0000,3.5714,20220513


In [7]:
for i in price_df.簡稱.unique():
    if len(i) == 1:
        print(i)

宏
建
啟
矬


In [8]:
price_df.loc[price_df['簡稱'] == '建', '簡稱'] = '建碁'
price_df.loc[price_df['簡稱'] == '宏', '簡稱'] = '宏碁'
price_df.loc[price_df['簡稱'] == '啟', '簡稱'] = '啟碁'
price_df.loc[price_df['簡稱'] == '矬', '簡稱'] = '恒耀'

In [9]:
for i in price_df.簡稱.unique():
    if len(i) == 1:
        print(i)

In [10]:
print(len(price_df['簡稱'].unique()))

1211


In [15]:
price_dic = {}
return_dic = {}

In [16]:
for i in price_df['簡稱'].unique():
    price_dic[i] = {}
    return_dic[i] = {}

for i in price_df['簡稱'].unique():
    singleCompany = price_df[price_df['簡稱'] == i].reset_index(drop=True)
    for j in singleCompany.index:
        price_dic[i][singleCompany.loc[j, '日期']] = singleCompany.loc[j, '收盤價(元)']
        return_dic[i][singleCompany.loc[j, '日期']] = singleCompany.loc[j, '報酬率％']


In [28]:
for i in list(price_dic):
    if i == clearSentence(i):
        continue
    else:
        price_dic[clearSentence(i)] = price_dic.pop(i)
        return_dic[clearSentence(i)] = return_dic.pop(i)

In [32]:
import json

os.chdir('/Users/arthur/Desktop/Event_Driven_Stock_Prediction_using_Deep_Learning/input')

with open("stockPrices.json", 'w') as outfile:
    json.dump(price_dic, outfile)
with open("stockReturns.json", 'w') as outfile:
    json.dump(return_dic, outfile)